In [ ]:
# Scrapping WOS v.1

from wos import WosClient
import wos.utils

with WosClient('emil.raducanu@cirad.fr', 'P9%8M$VPn$%ZmD#6PPYUG*EyRq*ThmZvW9zq6^Q57pRvFyD9dkUZ^M5rZiYz7bie96uCF@2&pr^PdgJC#LPHfc%ptoUk@@') as client:
    print(wos.utils.query(client, 'AU=Knuth Donald'))

In [ ]:
# Scrapping WOS v.2

import time
import clarivate.wos_starter.client
from pprint import pprint
from clarivate.wos_starter.client.apis.tags import documents_api
from clarivate.wos_starter.client.model.document import Document
from clarivate.wos_starter.client.model.documents_list import DocumentsList

In [ ]:
# Scrapping Sci-Hub

from scihub import SciHub

sh = SciHub()

# exactly the same thing as fetch except downloads the articles to disk
# if no path given, a unique name will be used as the file name
result = sh.download('https://onlinelibrary.wiley.com/doi/full/10.1111/gcb.12569', path='paper.pdf')